# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
import gradio as gr
import openai
import os
import json
from dotenv import load_dotenv

In [ ]:
# --- Configuration and Setup ---

# Load environment variables from a .env file (recommended for API keys)
load_dotenv()

# Set your OpenAI API key from an environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")

# The absolute path to the root directory the LLM is allowed to access.
# os.path.expanduser handles the '~' to find your home directory.
ALLOWED_DIRECTORY = os.path.expanduser("~/projects/pleasurewebsite/mysite")

# Ensure the allowed directory exists
if not os.path.isdir(ALLOWED_DIRECTORY):
    print(f"Error: The specified directory '{ALLOWED_DIRECTORY}' does not exist.")
    print("Please double-check the path to your project.")
    # Exit if the directory isn't found, as the app is useless without it.
    exit()


In [ ]:
# --- Tool Functions: The LLM's "Hands" ---

def list_files_in_directory(path="."):
    """
    Lists files and directories within a specified path inside the allowed directory.
    The path is relative to the project root.
    """
    try:
        # Security Check: Resolve the requested path and ensure it's within the allowed directory.
        base_path = ALLOWED_DIRECTORY
        requested_path = os.path.abspath(os.path.join(base_path, path))

        if not requested_path.startswith(base_path):
            return "Error: Access denied. Path is outside the allowed directory."

        if not os.path.isdir(requested_path):
            return f"Error: '{path}' is not a valid directory."

        items = os.listdir(requested_path)
        dirs = sorted([item for item in items if os.path.isdir(os.path.join(requested_path, item))])
        files = sorted([item for item in items if os.path.isfile(os.path.join(requested_path, item))])
        
        response = f"Contents of '{path}':\n\n"
        if dirs:
            response += "Directories:\n" + "\n".join(f"- {d}/" for d in dirs) + "\n\n"
        if files:
            response += "Files:\n" + "\n".join(f"- {f}" for f in files) + "\n"
        
        return response if items else f"The directory '{path}' is empty."

    except Exception as e:
        return f"An error occurred: {str(e)}"


In [ ]:
def read_file_content(file_path):
    """
    Reads the content of a specified file within the allowed directory.
    The file_path is relative to the project root.
    """
    try:
        base_path = ALLOWED_DIRECTORY
        requested_path = os.path.abspath(os.path.join(base_path, file_path))

        if not requested_path.startswith(base_path):
            return "Error: Access denied. File is outside the allowed directory."

        if not os.path.isfile(requested_path):
            return f"Error: File not found at '{file_path}'"

        with open(requested_path, 'r', encoding='utf-8', errors='ignore') as f:
            content = f.read(5000) 
            if len(content) == 5000:
                return content + "\n\n[... file content truncated ...]"
            return content

    except Exception as e:
        return f"An error occurred while reading the file: {str(e)}"


In [ ]:
# --- Tool Definitions for the OpenAI API ---

tools = [
    {
        "type": "function",
        "function": {
            "name": "list_files_in_directory",
            "description": "Get a list of files and directories in a specified path within the Django project. Use '.' for the root.",
            "parameters": { "type": "object", "properties": { "path": { "type": "string", "description": "The relative path to the directory. E.g., 'home/views'."}}, "required": ["path"]},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "read_file_content",
            "description": "Read the text content of a specific file within the Django project.",
            "parameters": { "type": "object", "properties": { "file_path": { "type": "string", "description": "The relative path to the file. E.g., 'mysite/settings.py'."}}, "required": ["file_path"]},
        },
    },
]

available_functions = {
    "list_files_in_directory": list_files_in_directory,
    "read_file_content": read_file_content,
}


In [ ]:
# --- Core Chat Logic ---

def chat_with_tools(user_message, history):
    """
    Handles the conversation, including tool calls and streaming the final response.
    """
    system_message = {
        "role": "system",
        "content": f"You are a helpful AI assistant with access to a local Django project file system. The root of the project is '{ALLOWED_DIRECTORY}'. You can list files and read their content. When asked about the project, use your tools to find the relevant information before answering. Be concise."
    }
    
    messages = [system_message]
    for human_msg, ai_msg in history:
        messages.append({"role": "user", "content": human_msg})
        messages.append({"role": "assistant", "content": ai_msg})
    messages.append({"role": "user", "content": user_message})

    try:
        first_response = openai.chat.completions.create(
            model="gpt-4-turbo",
            messages=messages,
            tools=tools,
            tool_choice="auto",
        )
        response_message = first_response.choices[0].message
        tool_calls = response_message.tool_calls

        if tool_calls:
            messages.append(response_message)
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                print(f"🤖 Calling tool: {function_name}({function_args})")
                function_response = function_to_call(**function_args)
                messages.append({"tool_call_id": tool_call.id, "role": "tool", "name": function_name, "content": function_response})
            
            final_stream = openai.chat.completions.create(model="gpt-4-turbo", messages=messages, stream=True)
            for chunk in final_stream:
                yield chunk.choices[0].delta.content or ""
        else:
            if response_message.content:
                 # If the model replies directly without tools, stream that reply.
                 # We need to simulate a stream from a non-streamed response.
                 yield response_message.content
            else:
                 yield "I am not sure how to answer that. Please try rephrasing."


    except Exception as e:
        print(f"An error occurred: {e}")
        yield "Sorry, an error occurred. Please check the console for details."


In [ ]:
# --- NEW: Gradio Interface with gr.Blocks for better layout ---

with gr.Blocks(theme="soft", title="Django Project Assistant 🤖") as demo:
    gr.Markdown("# Django Project Assistant 🤖")
    gr.Markdown("Ask me questions about your Django project. I can list files and read their contents to help you.")
    
    # The chatbot window is now taller to prevent text from disappearing.
    chatbot = gr.Chatbot(label="Conversation", height=600, bubble_full_width=False) 
    
    with gr.Row():
        # The scale parameter makes the textbox take up more horizontal space.
        msg_textbox = gr.Textbox(
            label="Your Message",
            placeholder="e.g., What files are in mysite/?",
            scale=7,
            autofocus=True,
        )
        submit_btn = gr.Button("Send", variant="primary", scale=1)

    # This function handles the entire chat interaction for the UI.
    def handle_chat(user_input, history):
        # Add the user's message to the chat display immediately.
        history.append([user_input, ""])
        
        # Prepare history for the backend function (which expects a list of tuples).
        formatted_history = history[:-1]
        
        # Get the streaming response from the core logic.
        response_stream = chat_with_tools(user_input, formatted_history)
        
        # Stream the response into the chatbot.
        for chunk in response_stream:
            history[-1][1] += chunk
            yield history, "" # Yield updated history and clear the textbox.
        
        # Final update after the stream is complete.
        yield history, ""

    # Event handlers for submitting the message.
    submit_btn.click(handle_chat, [msg_textbox, chatbot], [chatbot, msg_textbox])
    msg_textbox.submit(handle_chat, [msg_textbox, chatbot], [chatbot, msg_textbox])

    # Add example questions for the user.
    gr.Examples(
        examples=[
            ["What files are in the root directory?"],
            ["Can you show me the contents of 'mysite/settings.py'?"],
            ["What's inside the 'home/templates/home' directory?"]
        ],
        inputs=msg_textbox,
        outputs=[chatbot, msg_textbox],
        fn=handle_chat,
        cache_examples=False,
    )

In [ ]:
if __name__ == "__main__":
    demo.launch(debug=True)